In [1]:
# set-up (ne sera pas dans la présentation, seulement dans ce notebook):

import pandas as pd
import numpy as np
from functools import reduce

n = 1_000_000

n_dicts = 10_000
list_of_dicts = [
    dict(zip(np.random.choice(10**10, n // n_dicts), range(n)))
    for _ in range(n_dicts)
]

n_users = 100_000
df = pd.DataFrame({'user': np.random.choice(n_users, n), 'item': np.random.choice(100, n)})

n_groups = 100_000
x = np.random.rand(n)
group_idx = np.repeat(np.arange(n_groups), n // n_groups)
np.random.shuffle(group_idx)

(début de la présentation)

### Quel est le point commun de ces 3 fonctions?

In [2]:
def merge_dicts(list_of_dicts: list[dict]) -> dict:
    return reduce(lambda d1, d2: {**d1, **d2}, list_of_dicts)

In [3]:
def compute_n_uniques_by_user(df: pd.DataFrame) -> dict:
    n_uniques = {}
    for user in df['user'].unique():
        user_activity = df[df['user'] == user]
        n_uniques[user] = user_activity['item'].nunique()
    return n_uniques

In [4]:
def compute_avg_by_group(group_idx: np.ndarray[int], x: np.ndarray) -> np.ndarray:
    n_groups = group_idx.max() + 1
    avg_by_group = np.empty(n_groups)
    for idx in range(n_groups):
        avg_by_group[idx] = np.mean(x[group_idx == idx])
    return avg_by_group

Indice:

In [5]:
n = sum(len(d) for d in list_of_dicts)
n = len(df)
n = len(x)

### Réponse: L'inefficacité algorithmique! 

En effet, ces 3 petits algos peuvent être en $ O(n^2) $, selon les données.

Et pourtant j'ai vu ce pattern un grand nombre de fois:
- l'exemple avec les dictionnaires, je l'ai vue tel quel: une collègue m'a appelé parce qu'elle ne comprenait pas pourquoi c'était si lent.
- l'exemple avec pandas, c'est celui que j'ai vue le plus! Les data scientists l'emploient souvent lorsque ce qu'ils veulent faire n'est pas 
  facilement faisable avec des `groupby` et autres (ou pas faisable du tout, ça peut arriver).
- l'exemple avec numpy, ... bah je l'ai vue dans [scikit-learn](https://github.com/scikit-learn/scikit-learn/blob/473fef0bf83882efbc3273c2dfb4d82491e9066b/sklearn/ensemble/_gb.py#L257-L266)! Dans un cas où le $O(n^2)$ arrive pour des hyper-paramètres du modèles peu courants, mais qui doivent quand-même être explorés régulièrement dans des grid search ou similaire.


In [6]:
sum(len(d) for d in list_of_dicts), len(df), len(x)

(1000000, 1000000, 1000000)

In [7]:
%time merge_dicts(list_of_dicts)
%time compute_n_uniques_by_user(df)
%time compute_avg_by_group(group_idx, x)

CPU times: user 51.9 s, sys: 39.4 s, total: 1min 31s
Wall time: 1min 31s
CPU times: user 1min 21s, sys: 13.5 ms, total: 1min 21s
Wall time: 1min 21s
CPU times: user 1min 4s, sys: 4.82 ms, total: 1min 4s
Wall time: 1min 4s


array([0.46392534, 0.5279945 , 0.43661711, ..., 0.46826864, 0.48314751,
       0.65158763], shape=(100000,))

### Pourquoi on écrit ça quand-même?

Parce que lorsqu'on l'écrit, la complexité algorithmique est caché par la syntaxe.

Quand je demande au gens qui ont écrit ce pattern en pandas quelle est la complexité théorique de leur code d'après eux, ils me répondent souvent $O(n)$, car il ne process qu'une fois chaque ligne du dataframe. Ils passent à coté du fait que `df['user'] == user` process toutes les lignes du dataframe.

### Et du coup, comment on évite ça?

Déjà, il faut bien comprendre Python et les librairies qu'on utilise (pandas, numpy, ...).

Ensuite il faut connaître les patterns qui permettent d'éviter ça:


### Amélioration de l'exemple avec les dictionnaires:

In [8]:
merged_dict = {}
for d in list_of_dicts:
    merged_dict.update(d)  # O(len(d))

### Amélioration de l'exemple en pandas:

In [9]:
n_uniques_by_user = df.groupby('user')['item'].nunique()

Oui ok, mais si c'est plus compliqué ?

In [24]:
# assez lent, mais en O(n) donc pas de mauvaise surprise
_ = df.groupby('user')['item'].apply(lambda user_items: user_items.nunique(), include_groups=False)

C'est bien plus lent, car on paye un overhead* pour chaque groupe. Mais c'est aussi vrai pour la boucle qu'on avait avant, et là on évite le $O(n^2)$ qui sera juste impossible à faire tourner dès que $n$ va dans les millions.

*création d'un petit dataframe pour chaque groupe, ce qui implique de nombreuses ligne de Python, et une ligne de Python ça coûte cher

### Amélioration de l'exemple en numpy:

Une solution facile, c'est de passer par `pandas` + `groupby`.


In [11]:
avg_by_group = pd.Series(x).groupby(group_idx).mean()

Mais parfois on veut éviter pandas. Par exemple, pandas n'est pas une dépendance de scikit-learn, donc on veut résoudre ça sans pandas.

In [12]:
n_by_group = np.bincount(group_idx, minlength=n_groups)
sum_by_group = np.bincount(group_idx, weights=x, minlength=n_groups)

avg_by_group = sum_by_group / n_by_group

Ok, mais bincount ça fait seulement des sommes / des histogrammes, et si on veut faire plus? 

Par exemple le minimum par groupe:

In [13]:
min_by_group = np.full(n_groups, np.inf)
np.minimum.at(min_by_group, group_idx, x)

C'est quoi ce `.at` magique? C'est les méthodes des fonctions universelles de numpy (`ufunc`):
https://numpy.org/doc/stable/reference/ufuncs.html#methods


De nombreuses fonctions numpy sont des ufuncs, notamment: `add`, `minimum`, `maximum` et même `gcd`.

### Pour aller plus loin:

1) Que faire si je n'ai pas des indices denses dans $[0, n_{groups})$ mais des IDs arbitraires?

In [14]:
x = np.random.rand(n)
group_id = np.random.choice(np.random.choice(10**18, n_groups), n)

In [15]:
unique_ids, group_idx = np.unique(group_id, return_inverse=True)
# ^ O(n log n), `np.unique` trie son input
# En pratique, ce O(n log n) est 10-100x plus lent qu'un `bincount` par exemple,
# ce qui reste généralement bien assez rapide. Par exemple, cette cellule
# s'exécute en ~70ms pour 1M d'éléments
n_groups = unique_ids.size

sum_by_group = np.bincount(group_idx, weights=x)

2. Que faire si les ufuncs ne suffisent pas? Par exemple, ça ne permet pas de calculer la médiane par groupe.

In [16]:
from scipy.sparse import csr_array

In [27]:
x_per_group = csr_array((
    x,
    (group_idx, np.arange(n))
), shape=(n_groups, n))

In [18]:
def median_per_row(data, indptr):
    n_rows = indptr.size - 1
    out = np.full(n_rows, np.nan)
    for row in range(n_rows):
        row_start, row_end = indptr[row], indptr[row + 1]
        if row_start == row_end:
            continue
        out[row] = np.median(data[row_start:row_end])
    return out

_ = median_per_row(x_per_group.data, x_per_group.indptr)

Si on a besoin de vitesse, ce genre de fonctions est généralent compatible avec numba:

(si on était dans scikit-learn, on ferait plutôt pour une implémentation en Cython)

In [19]:
from numba import njit

median_per_row_numba = njit(median_per_row)
_ = median_per_row_numba(x_per_group.data, x_per_group.indptr)

In [20]:
%timeit _ = median_per_row(x_per_group.data, x_per_group.indptr)
%timeit _ = median_per_row_numba(x_per_group.data, x_per_group.indptr)

921 ms ± 46 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
23.1 ms ± 73.1 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [21]:
%timeit _ = pd.Series(x).groupby(group_idx).median()

47.2 ms ± 2.33 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
